In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [13]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 3)
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    open = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,kernel = np.ones((1,1),np.uint8))
    sure_bg = cv2.dilate(open, np.ones((2,2),np.uint8), iterations=1)
    dist_trans = cv2.distanceTransform(open, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_trans, 0.7*dist_trans.max(),255, cv2.THRESH_BINARY)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 2
    markers[unknown==255] = 0 
    st = cv2.watershed(frame, markers)
    frame[st == -1] = [0, 0, 255] 
    cv2.imshow('Test',frame)
    if cv2.waitKey(5) & 0xFF==27:
        break
cv2.destroyAllWindows()
cap.release()

In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Stop if the frame is not read correctly

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 3)
    
    # Apply adaptive thresholding
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Morphological operations
    kernel = np.ones((3, 3), np.uint8)  # Larger kernel for better noise removal
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    sure_bg = cv2.dilate(opening, kernel, iterations=2)

    # Distance transform to get sure foreground
    dist_trans = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_trans, 0.7 * dist_trans.max(), 255, cv2.THRESH_BINARY)

    # Convert to uint8
    sure_fg = np.uint8(sure_fg)
    
    # Finding unknown region
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add 2 to all labels to ensure background is not confused with foreground
    markers = markers + 2

    # Mark the unknown region with 0
    markers[unknown == 255] = 0

    # Apply watershed
    markers = cv2.watershed(frame, markers)
    frame[markers == -1] = [0, 0, 255]  # Mark boundaries in red

    # Show results
    cv2.imshow('Watershed Segmentation', frame)
    cv2.imshow('Thresholded Image', thresh)
    cv2.imshow('Distance Transform', dist_trans)

    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()
